In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
from geodatasets import get_path
from function_06 import load_data_with_delimiters
from shapely.geometry import Point
import folium
from folium.plugins import HeatMap


In [2]:
# Read all datas
file_info = "../data/raw/ongevallen.txt"
df_accidents =pd.read_csv(file_info, low_memory=False)

file_info2 = "../data/raw/partijen.txt"
df_A=pd.read_csv(file_info2, low_memory=False)

file_info3 = "../data/raw/wegvakgeografie_01012024.csv"
wegvak_df=pd.read_csv(file_info3, low_memory=False)

In [5]:
df_accidents.shape, df_accidents.isnull().sum(), df_accidents.columns

((1193484, 66),
 VKL_NUMMER          0
 REGNUMMER      197529
 PVOPGEM       1166161
 DATUM_VKL     1193484
 DAG_CODE      1193484
                ...   
 DISTRCODE      964941
 DISTRNAAM      964941
 DAGTYPE       1193484
 IND_ALC       1193484
 WEEKNR        1193484
 Length: 66, dtype: int64,
 Index(['VKL_NUMMER', 'REGNUMMER', 'PVOPGEM', 'DATUM_VKL', 'DAG_CODE',
        'MND_NUMMER', 'JAAR_VKL', 'TIJDSTIP', 'UUR', 'DDL_ID', 'AP3_CODE',
        'AP4_CODE', 'AP5_CODE', 'ANTL_SLA', 'ANTL_DOD', 'ANTL_GZH', 'ANTL_SEH',
        'ANTL_GOV', 'ANTL_PTJ', 'ANTL_TDT', 'MNE_CODE', 'AOL_ID', 'NIVEAUKOP',
        'WSE_ID', 'WSE_AN', 'BEBKOM', 'MAXSNELHD', 'WVL_ID', 'WVG_ID', 'WVG_AN',
        'WDK_ID', 'WDK_AN', 'LGD_ID', 'ZAD_ID', 'WGD_CODE_1', 'WGD_CODE_2',
        'BZD_ID_VM1', 'BZD_ID_VM2', 'BZD_ID_VM3', 'BZD_VM_AN', 'BZD_ID_IF1',
        'BZD_ID_IF2', 'BZD_ID_IF3', 'BZD_IF_AN', 'BZD_ID_TA1', 'BZD_ID_TA2',
        'BZD_ID_TA3', 'BZD_TA_AN', 'JTE_ID', 'WVK_ID', 'HECTOMETER', 'FK_VELD5',
       

In [7]:
df_accidents['WEEKNR'].unique()

array([nan])

In [9]:
relevant_columns = [
    'VKL_NUMMER', 'TIJDSTIP', 'UUR', 'JAAR_VKL',
    'ANTL_DOD', 'ANTL_GZH', 'ANTL_SEH',
    'WVK_ID', 'GME_NAAM', 'PLT_NAAM',
    'BZD_VM_AN', 'BZD_IF_AN', 'BZD_TA_AN'
]

# Keep only those columns
df_accidents = df_accidents[relevant_columns]

# Drop rows where critical identifiers or keys are missing
df_accidents = df_accidents.dropna(subset=['VKL_NUMMER', 'WVK_ID'])


In [11]:
df_accidents.shape

(864510, 13)

In [15]:
df_A.shape, df_A.head(), df_A['PTJ_ID'].head(), df_A.isnull().sum()

((1801034, 41),
      PTJ_ID   VKL_NUMMER  NUMMER  DOORRIJDER  OTE_ID OTE_AN  NTT_CODE_V  \
 0  25909704  20140010014       1         NaN    11.0    NaN         NaN   
 1  25909705  20140010014       2         NaN    64.0    NaN         NaN   
 2  25909706  20140010030       1         NaN     1.0    NaN         NaN   
 3  25909707  20140010030       2         NaN    61.0    NaN         NaN   
 4  25909708  20140010030       3         NaN    83.0    NaN         NaN   
 
    VTGVERZ  SCHADE  GETRAANH  ...  AGT_TYPE  AGT_ID_1  AGT_ID_2  BWG_ID_1  \
 0      NaN     NaN       NaN  ...         V      12.0       NaN      10.0   
 1      NaN     NaN       NaN  ...         V      12.0       NaN      10.0   
 2      NaN     NaN       NaN  ...         V      15.0       NaN      85.0   
 3      NaN     NaN       NaN  ...         V      12.0       NaN      10.0   
 4      NaN     NaN       NaN  ...       NaN       NaN       NaN       NaN   
 
    BWG_ID_2  BWG_AN  TDT_ID_1  TDT_ID_2  TDT_ID_3  TDT_

In [ ]:
# df_A = df_A.dropna(subset=['TDT_AN'])
# df_A = df_A.dropna(subset=['OTE_AN'] )

In [23]:
# Keywords related to bikes in OTE_AN
bike_keywords_ote = [
    'FIETS', 'E-BIKE', 'SPEED PEDELEC', 'SPEEDPEDELEC', 'FIETSKAR',
    'FIETSAANHANGER', 'DRIEWIELER', 'HANDBIKE', 'FATBIKE', 'FAT BIKE',
    'SKELTER', 'BUGGY', 'ELEKTRISCHE-STEP', 'ELEKTRISCHE STEP', 'ELEKTRISCHE DRIEWIELER'
]

# Keywords indicating cyclist-related circumstances in TDT_AN
bike_keywords_tdt = [
    'FIETS', 'FIETSPAD', 'EVENWICHT', 'PORTIER OPENEN', 'GEEN VERLICHTING',
    'OBSTAKEL', 'SLECHT WEGDEK', 'AUTO OP FIETSPAD', 'ONVOORZICHTIG OP WEG'
]


bike_accidents = df_A[(df_A['OTE_AN'].isin(bike_keywords_ote)) | (df_A['TDT_AN'].isin(bike_keywords_tdt))]

# Filter rows where either column contains one of the keywords (case-insensitive)
#bike_accidents = df_A[
#    df_A['OTE_AN'].str.contains('|'.join(bike_keywords_ote), case=False, na=False) |
#    df_A['TDT_AN'].str.contains('|'.join(bike_keywords_tdt), case=False, na=False)
#]


In [25]:
bike_accidents['OTE_AN'].unique()

array([nan, 'FIETSAANHANGER', 'SKELTER', 'DRIEWIELER', 'HANDBIKE',
       'ELEKTRISCHE STEP', 'BUGGY', 'ELEKTRISCHE DRIEWIELER',
       'SPEED PEDELEC', 'FATBIKE', 'SPEEDPEDELEC', 'FIETSKAR', 'FAT BIKE',
       'ELEKTRISCHE-STEP'], dtype=object)

In [27]:
bike_accidents['TDT_AN'].unique()


array(['PORTIER OPENEN', nan, 'GEEN VERLICHTING', 'SLECHT WEGDEK',
       'OBSTAKEL', 'ONVOORZICHTIG OP WEG', 'AUTO OP FIETSPAD'],
      dtype=object)

In [29]:
bike_accidents = bike_accidents[['PTJ_ID', 'VKL_NUMMER', 'NUMMER','OTE_AN','TDT_AN']]


In [31]:
bike_accidents.head()

,PTJ_ID,VKL_NUMMER,NUMMER,OTE_AN,TDT_AN
32328,26052294,20140096088,1,NaN,PORTIER OPENEN
60168,26084674,20140113085,3,FIETSAANHANGER,NaN
66861,26093279,20140117493,2,SKELTER,NaN
74127,26083750,20140112604,1,NaN,PORTIER OPENEN
77081,26061609,20140101008,3,FIETSAANHANGER,NaN


In [33]:
bike_accidents.isnull().sum()

PTJ_ID          0
VKL_NUMMER      0
NUMMER          0
OTE_AN          8
TDT_AN        387
dtype: int64

In [35]:
# Filter the wegvak dataset for rows where the municipality is Amsterdam
wegvak_amsterdam = wegvak_df[wegvak_df['gme_naam'].str.lower() == 'amsterdam']

# Check how many rows and preview a few
wegvak_amsterdam.shape, wegvak_amsterdam[['wvk_id', 'gme_naam', 'shape']].head()


((38887, 42),
         wvk_id   gme_naam                                              shape
 10   239374087  Amsterdam  LINESTRING (119808.14400000125 487129.29500000...
 31   600416574  Amsterdam  LINESTRING (120287.60770000145 485849.17119999...
 258  234376081  Amsterdam  LINESTRING (117476.56010000035 488051.62629999...
 264  231369048  Amsterdam  LINESTRING (115864.78000000119 484508.03849999...
 265  230374012  Amsterdam  LINESTRING (115203.02639999986 487453.90980000...)

In [37]:
columns_to_keep = [
    'wvk_id',           # Road Segment ID
    'wvk_begdat',       # Start Date
    'wvk_enddat',       # End Date (optional if not always available)
    'wegnummer',        # Road Number
    'stt_naam',         # Street Name
    'wpsnaam',          # Neighborhood Name
    'gme_naam',         # Municipality Name
    'rijrichtng',       # Driving Direction
    'wegbehsrt',        # Road Type / Category
    'shape'             # shape
]


In [39]:
wegvak_amsterdam = wegvak_amsterdam[columns_to_keep]

# Drop rows with NaN in critical columns
#wegvak_amsterdam = wegvak_amsterdam.dropna(subset=['wvk_id', 'wegnummer', 'stt_naam', 'shape'])


In [41]:
# Ensure column name consistency for the join
bike_data_geo = df_accidents.copy()

# Confirm WVK_ID exists in both datasets
if 'WVK_ID' in bike_data_geo.columns and 'wvk_id' in wegvak_amsterdam.columns:
    # Merge on WVK_ID
    bike_amsterdam_geo = bike_data_geo.merge(
        wegvak_amsterdam,
        left_on='WVK_ID',
        right_on='wvk_id',
        how='inner'
    )
    match_found = True
else:
    bike_amsterdam_geo = pd.DataFrame()
    match_found = False

# Output results
# match_found, bike_amsterdam_geo[['Accident_ID', 'Vehicle_Type', 'Municipality', 'wvk_id', 'shape']].head() if match_found else "No matches found"


In [43]:
bike_amsterdam_geo['shape'].head()


0    LINESTRING (121614.72899999842 487348.85099999...
1    LINESTRING (120977 492472.8960000016, 120978.0...
2    LINESTRING (117115.5390000008 489289.030900001...
3    LINESTRING (118986.91569999978 483450.53599999...
4    LINESTRING (121405.81069999933 483628.85469999...
Name: shape, dtype: object

In [45]:
bike_accidents.head()

,PTJ_ID,VKL_NUMMER,NUMMER,OTE_AN,TDT_AN
32328,26052294,20140096088,1,NaN,PORTIER OPENEN
60168,26084674,20140113085,3,FIETSAANHANGER,NaN
66861,26093279,20140117493,2,SKELTER,NaN
74127,26083750,20140112604,1,NaN,PORTIER OPENEN
77081,26061609,20140101008,3,FIETSAANHANGER,NaN


In [47]:
bike_data = bike_accidents.merge(bike_amsterdam_geo, on='VKL_NUMMER', how='outer')


In [49]:
bike_data.shape

(30149, 27)

In [51]:
bike_data.isnull().sum()

PTJ_ID        29754
VKL_NUMMER        0
NUMMER        29754
OTE_AN        29762
TDT_AN        30141
TIJDSTIP      30149
UUR           30149
JAAR_VKL        384
ANTL_DOD      30149
ANTL_GZH      30149
ANTL_SEH      30149
WVK_ID          384
GME_NAAM        384
PLT_NAAM        384
BZD_VM_AN     30144
BZD_IF_AN     28208
BZD_TA_AN     30041
wvk_id          384
wvk_begdat      384
wvk_enddat     9558
wegnummer     18235
stt_naam        384
wpsnaam         384
gme_naam        384
rijrichtng      384
wegbehsrt       384
shape           384
dtype: int64

In [53]:
# Drop 'WVK_ID' (duplicate of 'wvk_id') and 'Unnamed: 0' (auto index column)
#bike_data = bike_data.drop(columns=['WVK_ID'])
rename_dict = {
    'PTJ_ID': 'Participant_ID',
    'VKL_NUMMER': 'Accident_ID',
    'NUMMER': 'Case_Number',
    'OTE_AN': 'Object_Involved',
    'TDT_AN': 'Contributing_Factor',
    'TIJDSTIP': 'Time',
    'UUR': 'Hour',
    'JAAR_VKL': 'Accident_Year',
    'ANTL_DOD': 'Fatalities',
    'ANTL_GZH': 'Severe_Injuries',
    'ANTL_SEH': 'Emergency_Room_Injuries',
    'WVK_ID': 'RoadSegment_ID',
    'GME_NAAM': 'Municipality',
    'PLT_NAAM': 'Police_Area',
    'BZD_VM_AN': 'Vehicle_Type',
    'BZD_IF_AN': 'Object_Collided_With',
    'BZD_TA_AN': 'Other_Party_Involved',
    'wvk_id': 'RoadSegment_ID',  # duplicate
    'wvk_begdat': 'Segment_StartDate',
    'wvk_enddat': 'Segment_EndDate',
    'wegnummer': 'RoadNumber',
    'stt_naam': 'StreetName',
    'wpsnaam': 'Neighborhood',
    'gme_naam': 'Municipality',  # duplicate
    'rijrichtng': 'DrivingDirection',
    'wegbehsrt': 'RoadType',
    'shape': 'Shape'
}

# Step 3: Optional rename for clarity (if needed)
bike_data = bike_data.rename(columns=rename_dict)

In [55]:
# Display updated column names
bike_data.columns.tolist()

['Participant_ID',
 'Accident_ID',
 'Case_Number',
 'Object_Involved',
 'Contributing_Factor',
 'Time',
 'Hour',
 'Accident_Year',
 'Fatalities',
 'Severe_Injuries',
 'Emergency_Room_Injuries',
 'RoadSegment_ID',
 'Municipality',
 'Police_Area',
 'Vehicle_Type',
 'Object_Collided_With',
 'Other_Party_Involved',
 'RoadSegment_ID',
 'Segment_StartDate',
 'Segment_EndDate',
 'RoadNumber',
 'StreetName',
 'Neighborhood',
 'Municipality',
 'DrivingDirection',
 'RoadType',
 'Shape']

In [57]:
bike_data.columns = bike_data.columns.str.lower()


In [59]:
bike_data.isnull().sum()

participant_id             29754
accident_id                    0
case_number                29754
object_involved            29762
contributing_factor        30141
time                       30149
hour                       30149
accident_year                384
fatalities                 30149
severe_injuries            30149
emergency_room_injuries    30149
roadsegment_id               384
municipality                 384
police_area                  384
vehicle_type               30144
object_collided_with       28208
other_party_involved       30041
roadsegment_id               384
segment_startdate            384
segment_enddate             9558
roadnumber                 18235
streetname                   384
neighborhood                 384
municipality                 384
drivingdirection             384
roadtype                     384
shape                        384
dtype: int64

In [61]:
df_ams_cc = bike_data.drop(columns=['contributing_factor','roadtype', 'drivingdirection', 'case_number','object_involved','police_area', 'time', 'hour','other_party_involved','object_collided_with','emergency_room_injuries','severe_injuries','fatalities', 'vehicle_type'], errors='ignore')


In [ ]:
df_ams_cc.to_csv('../data/cleaned/df_ams_cc.csv')

In [ ]:
bike_data.to_csv('../data/cleaned/df_ams_ccenter.csv')